In [1]:
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import concurrent.futures
import requests

# Replace with your actual API keys for Bybit, Bitget, and OKX
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
OKX_API_KEY = '48af790d-3250-44ec-9528-54d5cd1c1009'
OKX_API_SECRET = 'C91F9F5826FD2F5A8ADC91D945643AD9'
OKX_API_PASSWORD = 'OkX!?Secure789Trade2025?!'

# Telegram and Discord Bot Settings
TELEGRAM_BOT_TOKEN = '7570092528:AAEe-BPQH5LqRq76vxDlxdtsFvhYlU6y7yg'
TELEGRAM_CHAT_ID = '8010042439'
DISCORD_WEBHOOK_URL = 'https://discord.com/api/webhooks/1354874948845568170/_T8lTTaa_w26N0tZr1QJYFb8YkIXB7ct89U_pUuI2ycVEaqtu5WTf9FakHacAkjGJQ-O'

# Alert Functions
def send_telegram_alert(message):
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
        requests.post(url, data=data)
    except Exception as e:
        print(f"❌ Telegram Error: {e}")

def send_discord_alert(message):
    try:
        requests.post(DISCORD_WEBHOOK_URL, json={"content": message})
    except Exception as e:
        print(f"❌ Discord Error: {e}")

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to OKX API
okx = ccxt.okx({
    'apiKey': OKX_API_KEY,
    'secret': OKX_API_SECRET,
    'password': OKX_API_PASSWORD,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% for testing
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")  # Change this to your local timezone

# Adjustable trading fees per exchange
TRADING_FEES = {
    'Bybit': 0.001,
    'Bitget': 0.0008,
    'OKX': 0.0007
}

exchanges = {'Bybit': bybit, 'Bitget': bitget, 'OKX': okx}

# Cache futures markets
futures_markets_cache = {}

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]

        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if exchange_name == "OKX" and market.get('type') != 'spot':
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        return ticker.get("last", None)
    except Exception:
        return None

def fetch_prices_concurrently(exchange, symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(exchange, symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_prices, futures_prices):
    spot_price = spot_prices.get(symbol)
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)

    if not futures_symbol:
        return

    futures_price = futures_prices.get(futures_symbol)
    if not spot_price or not futures_price:
        return

    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)

    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    ⚡ Execute arbitrage on {exchange_name}")
        print("-" * 60)

        alert = f"🔍 [{timestamp}] Arbitrage on {exchange_name}: {symbol}\nSpot: {spot_price:.5f} | Futures: {futures_price:.5f} | Spread: {spread:.4%} | Net: {net_profit:.4%}"
        send_telegram_alert(alert)
        send_discord_alert(alert)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbols = [find_futures_symbol(t, futures_list) for t in tokens if find_futures_symbol(t, futures_list)]

    spot_prices = fetch_prices_concurrently(exchange, tokens)
    futures_prices = fetch_prices_concurrently(exchange, futures_symbols)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_intra_exchange_arbitrage, exchange, exchange_name, token, spot_prices, futures_prices) for token in tokens]
        concurrent.futures.wait(futures)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

